In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random, gc, keras, os

from keras import backend as K
from keras.utils import plot_model
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16

%matplotlib inline

Using TensorFlow backend.


# Define Parameters

In [2]:
# Implement pre-trained VGG16 CNN model
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
plot_model(vgg16, show_shapes=True, to_file='./result/vgg16.pdf')
plot_model(vgg16, show_shapes=True, to_file='./result/vgg16.png')
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# Define Triplet Network

In [3]:
# Define base network for triplet network
def base_net(input_shape=(224, 224, 3), trainable=False):
    """ define triplet network """
    # load pre-trained VGG16 model
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg16.trainable = trainable
    
    # define sequential model
    model = Sequential(name='base_net')
    model.add(vgg16)
    model.add(Flatten(name='flatten'))
    model.add(Dense(512, activation='relu', name='fc1'))
    model.add(Dense(128, activation=None, name='fc2'))
    model.add(Lambda(lambda x: K.l2_normalize(x, axis=1), name='l2_norm'))
    
    return model

In [5]:
base_model = base_net(input_shape=(224, 224, 3), trainable=False)
plot_model(base_model, show_shapes=True, to_file='./result/base_model.pdf')
plot_model(base_model, show_shapes=True, to_file='./result/base_model.png')
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
fc1 (Dense)                  (None, 512)               12845568  
_________________________________________________________________
fc2 (Dense)                  (None, 128)               65664     
_________________________________________________________________
l2_norm (Lambda)             (None, 128)               0         
Total params: 27,625,920
Trainable params: 12,911,232
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Define triplet network
def triplet_net(base_model, input_shape=(224, 224, 3)):
    """ function to define triplet networks """
    # define input: anchor, positive, negative
    anc_input = Input(shape=input_shape, name='anchor_input')
    pos_input = Input(shape=input_shape, name='positive_input')
    neg_input = Input(shape=input_shape, name='negative_input')
    
    # extract vector represent using CNN based model
    anc_output = base_model(anc_input)
    pos_output = base_model(pos_input)
    neg_output = base_model(neg_input)
    
    # extract vector represent using CNN based model
    anc_output = Lambda(lambda x: x, name='anchor_output')(anc_output)
    pos_output = Lambda(lambda x: x, name='positive_output')(pos_output)
    neg_output = Lambda(lambda x: x, name='nagative_output')(neg_output)
    
    # define inputs and outputs
    inputs=[anc_input, pos_input, neg_input]
    outputs=[anc_output, pos_output, neg_output]
    
    # define the triplet model
    model = Model(inputs=inputs, outputs=outputs, name='triplet_net')
    
    return model

In [7]:
triplet_model = triplet_net(base_model=base_model, input_shape=(224, 224, 3))
plot_model(triplet_model, show_shapes=True, to_file='./result/triplet_network.pdf')
plot_model(triplet_model, show_shapes=True, to_file='./result/triplet_network.png')
triplet_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
base_net (Sequential)           (None, 128)          27625920    anchor_input[0][0]               
                                                                 positive_input[0][0]             
          

In [8]:
# Define triplet loss
def triplet_loss(y_true, y_pred):
    """ function to compute triplet loss
        margin is predefined coded, manually change if needed
    """
    # define triplet margin
    margin = 0.2
    
    # get the prediction vector
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # compute distance
    pos_distance = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    neg_distance = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    
    # compute loss
    partial_loss = tf.subtract(pos_distance, neg_distance) + margin
    full_loss = tf.reduce_sum(tf.maximum(partial_loss, 0.0))
    
    return full_loss